## Introduction to `robotoc` 
# 5: Optimal Control of a Humanoid

## The goal of this chapter
Here, we learn how to construct and solve an optimal control problem (OCP) for a humanoid robot iCub.

First, construct `robotoc.Robot` of a humanoid robot iCub.

In [ ]:
import robotoc
contact_frames = ['l_sole', 'r_sole']
contact_types = [robotoc.ContactType.SurfaceContact for _ in contact_frames]
baumgarte_time_step = 0.05
path_to_urdf = 'urdf/icub_description/urdf/icub.urdf'
icub = robotoc.Robot(path_to_urdf, robotoc.BaseJointType.FloatingBase, 
                     contact_frames, contact_types, baumgarte_time_step)

## Cost Function
Next, we create the cost function of the optimal control problem (OCP).   
In `robotoc`, we create the cost function `robotoc.CostFunction` (`std::shared_ptr<robotoc::CostFunction>` in C++) by combining various **cost components**.  
That is, `robotoc.CostFunction` (`std::shared_ptr<robotoc::CostFunction>` in C++) is a collection of the **cost components** .   
Each **cost component** inherits `robotoc.CostFunctionComponentBase` (`robotoc::CostFunctionComponentBase` in C++).    
By doing so, we can easily combine many kinds of **cost components**.

Basic and useful **cost components** are already implemented in `robotoc` .   
In this example, we utilize the cost **component on the configuration (i.e., cost on the joint-space)**.

### Jumping Pattern
Before constructing the cost function, we define parameters for the jumping pattern.   
Here, we consider a dynamic jumping of the robot.

In [ ]:
import numpy as np
jump_length = np.array([0.5, 0, 0])
flying_time = 0.25
ground_time = 0.7

### Configuration Space Cost
First, we define a configuration-space cost component.  
We set the reference values and weight parameters of the configuration space cost.  
Naming conventions are as follows:
- `q_ref`: reference configuration (joint positions)
- `q_weight`: weight on the configuration 
- `v_weight`: weight on the generalized velocity (joint velocities)
- `a_weight`: weight on the generalized acceleration (joint accelerations)
- `u_weight`: weight on the joint torques
- `q_weight_terminal`, `v_weight_terminal`: weights on the configuration and generalized velocity at the terminal time of the horizon
- `q_weight_impulse`, `v_weight_impulse`: weights on the configuration and generalized velocity at the impulse instant (just before the impact, i.e., touch-down of the feet onto the ground)
- `dv_weight_impulse`: weight on the impulse change in the generalized velocity at the impulse stage due to the impact

In [ ]:
q_standing = np.array([0, 0, 0.592, 0, 0, 1, 0,
                       0.20944, 0.08727, 0, -0.1745, -0.0279, -0.08726, # left leg
                       0.20944, 0.08727, 0, -0.1745, -0.0279, -0.08726, # right leg
                       0, 0, 0, # torso
                       0, 0.35, 0.5, 0.5, 0, 0, 0, # left arm 
                       0, 0.35, 0.5, 0.5, 0, 0, 0]) # right arm 
q_ref = q_standing.copy()
q_ref[0:3] += jump_length
q_weight = np.array([0, 1, 1, 100, 100, 100, 
                     0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 
                     0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 
                     0.001, 1, 1,
                     0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 
                     0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001])
v_weight = np.full(icub.dimv(), 1.0e-03)
a_weight = np.full(icub.dimv(), 1.0e-05)
q_weight_terminal = q_weight
v_weight_terminal = v_weight
q_weight_impulse = q_weight
v_weight_impulse = v_weight
config_cost = robotoc.ConfigurationSpaceCost(icub)
config_cost.set_q_ref(q_standing)
config_cost.set_q_weight(q_weight)
config_cost.set_v_weight(v_weight)
config_cost.set_a_weight(a_weight)
config_cost.set_q_weight_terminal(q_weight_terminal)
config_cost.set_v_weight_terminal(v_weight_terminal)
config_cost.set_q_weight_impulse(q_weight_impulse)
config_cost.set_v_weight_impulse(v_weight_impulse)

We then make the **cost function** `robotoc.CostFunction` and add the **configuration cost component** to it.

In [ ]:
cost = robotoc.CostFunction()
cost.push_back(config_cost)

## Constraints 
We can construct constraints in the same way as the cost function.  
In `robotoc`, we create the constraints `robotoc.Constraints` (`std::shared_ptr<robotoc::Constraints>` in C++) by combining various **constraint components**.  
That is, the `robotoc.Constraints` (`std::shared_ptr<robotoc::Constraints>` in C++) is a collection of the **constraint components** .  
Each **constraints component** inherits `robotoc.ConstraintComponentBase` (`robotoc::ConstraintComponentBase` in C++).   
By doing so, we can easily combine many kinds of **constraint components**.

Basic and useful **constraint components** are already implemented in robotoc.  
In this example, we utilize **constraint components on the joint position, velocity, and torque limits**.

In [ ]:
joint_position_lower  = robotoc.JointPositionLowerLimit(icub)
joint_position_upper  = robotoc.JointPositionUpperLimit(icub)
joint_velocity_lower  = robotoc.JointVelocityLowerLimit(icub)
joint_velocity_upper  = robotoc.JointVelocityUpperLimit(icub)
joint_torques_lower   = robotoc.JointTorquesLowerLimit(icub)
joint_torques_upper   = robotoc.JointTorquesUpperLimit(icub)

We also use the friction cone constraint component in this example.  

In [ ]:
friction_cone = robotoc.FrictionCone(icub)

We then make the **constraints** `robotoc.Constraints` and add these **constraint components**.

In [ ]:
constraints = robotoc.Constraints()
constraints.push_back(joint_position_lower)
constraints.push_back(joint_position_upper)
constraints.push_back(joint_velocity_lower)
constraints.push_back(joint_velocity_upper)
constraints.push_back(joint_torques_lower)
constraints.push_back(joint_torques_upper)
constraints.push_back(friction_cone)

## Contact Sequence
The **contact sequence** (`robotoc.ContactSequence`) describes the sequence of the **contact status** (`robotoc.ContactStatus`), which containts contact information such as which contacts are active at each time and contact placements (i.e., positions and rotations of the contacts).   
We set the contact sequence to the OCP (optimal control problem) solver by passing `robotoc.ContactSequence` .   
Then the OCP solver automatically constructs an OCP in which the robot dynamics follow the input contact sequence.

In constructing the contact sequence, we have to specify the maximum possible number of the discrete events (i.e., touch-down and lift-off in this example).   
In this example, we consider two jumps.   
We construct the contact sequence as 

In [ ]:
contact_sequence = robotoc.ContactSequence(icub)

We sequentially build the contact sequence of the biped walking.  
First, we make `robotoc.ContactStatus` object.   
We set which contacts are active and contact placements in this object.  
In addition, we set the friction coefficients that are used in the friction cone constraints.  
At the beginning, the robot is standing whose configuration is given as `q_standing` .  
This is specified as follows.

In [ ]:
icub.forward_kinematics(q_standing)
x3d0_L = icub.frame_placement('l_sole')
x3d0_R = icub.frame_placement('r_sole')
contact_placements = {'l_sole': x3d0_L, 'r_sole': x3d0_R} 
mu = 0.6
friction_coefficients = {'l_sole': mu, 'r_sole': mu} 
contact_status_standing = icub.create_contact_status()
contact_status_standing.activate_contacts(['l_sole', 'r_sole'])
contact_status_standing.set_contact_placements(contact_placements)
contact_status_standing.set_friction_coefficients(friction_coefficients)

We initialize the contact sequence with this contact status.

In [ ]:
contact_sequence.init(contact_status_standing)

Next, we add the contact sequence in which the contacts in two feet are inactive (that is, the robot is flying).    
This is expressed as follows.

In [ ]:
contact_status_flying = icub.create_contact_status()

We add (push_back) this contact status into the contact sequence.  
In this example, **we optimize the switching times (i.e., contact timings) as well as the trajectory (i.e., the switching time optimization (STO) problems)**.  
We specify this in the contact sequence as

In [ ]:
contact_sequence.push_back(contact_status_flying, ground_time, sto=True)

The input switching time is considreded as the initial guess of the switching time.


In the next phase, the both feet have active contacts.  
Now the position of two feet went forward with `jump_length` .   
This is described as follows.

In [ ]:
contact_placements['l_sole'].trans = contact_placements['l_sole'].trans + jump_length
contact_placements['r_sole'].trans = contact_placements['r_sole'].trans + jump_length 
contact_status_standing.set_contact_placements(contact_placements)
contact_sequence.push_back(contact_status_standing, ground_time+flying_time, sto=True)

Next, the robot again flying, i.e., the contacts in two feet are inactive.   
This is written as 

In [ ]:
contact_sequence.push_back(contact_status_flying, 2*ground_time+flying_time, sto=True)

Finally, the two feet become having contacts again.   
The both feet went forward with jump_length.

In [ ]:
contact_placements['l_sole'].trans = contact_placements['l_sole'].trans + jump_length 
contact_placements['r_sole'].trans = contact_placements['r_sole'].trans + jump_length 
contact_status_standing.set_contact_placements(contact_placements)
contact_sequence.push_back(contact_status_standing, 2*ground_time+2*flying_time, sto=True)

## STO (Switching Time Optimization) Cost and Constraints 
Since we consider the **STO problem** in this example, we have to define the STO cost (`robotoc.STOCostFunction`) and STO constraints (`robotoc.STOConstraints`).   
Otherwise, the solver does not try to solve the STO problems (just solves the OCP with fixed the switching times).

First, we define the STO cost, i.e., `robotoc.STOCostFunction` (`std::shared_ptr<robotoc::STOCostFunction>` in C++).   
We do not add any cost components in the cost.

In [ ]:
sto_cost = robotoc.STOCostFunction()

Next, we define the STO constraints, i.e., `robotoc.STOConstraints` (`std::shared_ptr<robotoc::STOConstraints>` in C++).   
In this object, we set the **minimum dwell-time**, i.e., minimum time-interval of each contact phase.   
It is important to set appropriate minimum dwell-times.  
Here, we have 4 switches and 5 phases.  
We define the STO constraints as follows.

In [ ]:
sto_constraints = robotoc.STOConstraints(min_dt=[0.6, 0.2, 0.6, 0.2, 0.6])

## Optimal Control Problem (OCP) and Solver

Now we can construct the optimal control problem (OCP) and solvers.  
First, set the length of the horizon `T` and the number of discretization grids `N` as 

In [ ]:
dt = 0.02
T = 2*flying_time + 3*ground_time
N = int(np.floor(T/dt))

Since this robot involves contacts and a floating base, we construct `robotoc.OCP` (`robotoc::OCP` in C++).   

In [ ]:
ocp = robotoc.OCP(robot=icub, cost=cost, constraints=constraints, 
                  sto_cost=sto_cost, sto_constraints=sto_constraints, 
                  contact_sequence=contact_sequence, T=T, N=N)

Then makes the OCP solver `robotoc.OCPSolver` (`robotoc::OCPSolver` in C++) for this OCP.  
This solver has the following characteristics: 
- Direct multiple-shooting method and primal-dual interior point method.
- Lifted-contact dynamics for robotic systems with contacts.
- Constraint-transformation for pure-state equality constraints.
- Riccati recursion to compute the Newton steps.
- Switching time optimization (STO) via Riccati recursion.

We can specify some solver options by `robotoc.SolverOptions`.    
In this example, we set
- `kkt_tol_mesh`: KKT tolerance for the mesh-refinement
- `max_dt_mesh`: Threshold of the maximum discretization size for the mesh-refinement
- `initial_sto_reg_iter`: Initial number of iteraions set by this value are regularized for the STO problem.
- `max_iter`: Maximum number of the iterations.
We also specify the number of threads used in parallel computing in the direct multiple-shooting method.

In [ ]:
solver_options = robotoc.SolverOptions()
solver_options.kkt_tol_mesh = 0.1
solver_options.max_dt_mesh = T/N 
solver_options.max_iter = 300
solver_options.initial_sto_reg_iter = 10 
ocp_solver = robotoc.OCPSolver(ocp=ocp, solver_options=solver_options, nthreads=4)

## Solve the OCP

We solve the OCP in which the initial state (`q`, `v`) and initial time (`t`) are given as follows.

In [ ]:
t = 0.
q = q_standing.copy()
v = np.zeros(icub.dimv())

We initialize the OCP solver.  
The following appropriate initialization is very simple but important to solve the OCP efficiently.   
First, we set the initial guess of the solution as the above `q` and `v`.   
Second, we set the initial guess of the solution of contact force `f` by the total weight of the robot.

In [ ]:
ocp_solver.set_solution("q", q)
ocp_solver.set_solution("v", v)
f_init = np.array([0.0, 0.0, 0.5*icub.total_weight()])
ocp_solver.set_solution("f", f_init)

We also initialize the solver to treat the inequality constraints.   
The following methods do it including the STO constraints. (Specifically, it initializes slack and dual variables of the primal-dual interior point method).

In [ ]:
ocp_solver.mesh_refinement(t)

To see how the current iterate is close to the stationary points, we see the **KKT error**, which is an l2-norm of the Karush–Kuhn–Tucker (KKT) conditions (first-order necessary conditions of the optimality).   
After the above initialization, the KKT error is 

In [ ]:
ocp_solver.KKT_error(t, q, v)

Now we solve the OCP

In [ ]:
ocp_solver.solve(t, q, v)

We can see that the KKT error is very small. (Specifically, smaller than the threshold set in `robotoc.SolverOption`).

In [ ]:
ocp_solver.KKT_error(t, q, v)

In [ ]:
stats = ocp_solver.get_solver_statistics()
print(stats)

We can also see the stats of the solver as follows.

In [ ]:
plot_kkt = robotoc.utils.PlotConvergence()
plot_kkt.figsize = 12, 9
plot_kkt.plot(kkt_data=ocp_solver.get_solver_statistics().kkt_error, 
              ts_data=ocp_solver.get_solver_statistics().ts)

We can also visualize the solution trajectory.  
If we choose `'gepetto'` as the `viewer_type`, we can also see the contact forces and friction cones.

In [ ]:
viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf, 
                                        base_joint_type=robotoc.BaseJointType.FloatingBase,
                                        viewer_type='gepetto')
viewer.set_contact_info(icub.contact_frames(), mu)
discretization = ocp_solver.get_time_discretization()
viewer.display(discretization.time_steps(), ocp_solver.get_solution('q'), 
               ocp_solver.get_solution('f', 'WORLD'))